In [3]:
!pip install transformers torch pandas numpy matplotlib bs4 langchain faiss-cpu
!pip install tiktoken
!pip install openai langchain faiss-cpu
!pip install langchain-community langchain-core
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 26.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 35.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 11.2 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 44.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
import os
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

from langchain.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_community.chat_models import ChatOpenAI

In [6]:
import requests
import bs4

url = "https://eman-physics.net"
resp = requests.get(url)
soup = bs4.BeautifulSoup(resp.text, 'html.parser')
text = soup.get_text()
chapters = []
for a_tag in soup.find_all('a', href=True):
    if 'contents' in a_tag['href']:
        chapter = a_tag['href'].split('/')[1]
        chapters.append(chapter)
chapters.remove('columns')

In [8]:
chapters

['math',
 'dynamics',
 'electromag',
 'analytic',
 'thermo',
 'statistic',
 'relativity',
 'quantum',
 'elementary',
 'fluid',
 'circuit',
 'chemistry']

In [9]:
# Fetch the webpage content
url_ = "https://eman-physics.net/"
links = []
for chapter in chapters:
    url = url_ + chapter + '/contents.html'
    response = requests.get(url)
    html_content = response.text

    # Use BeautifulSoup to parse the HTML content
    soup = bs4.BeautifulSoup(html_content, "html.parser")

    # Extract all  tags with href attributes
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if not href.startswith('http'):
            href = "https://eman-physics.net/" + chapter + href[1:]
            links.append(href)

In [10]:
links

['https://eman-physics.net/math/policy.html',
 'https://eman-physics.net/math/errors.html',
 'https://eman-physics.net/math/figures.html',
 'https://eman-physics.net/math/figures2.html',
 'https://eman-physics.net/math/figures3.html',
 'https://eman-physics.net/math/figures4.html',
 'https://eman-physics.net/math/semilog.html',
 'https://eman-physics.net/math/loglog.html',
 'https://eman-physics.net/math/linear01.html',
 'https://eman-physics.net/math/linear02.html',
 'https://eman-physics.net/math/linear03.html',
 'https://eman-physics.net/math/linear04.html',
 'https://eman-physics.net/math/linear05.html',
 'https://eman-physics.net/math/linear06.html',
 'https://eman-physics.net/math/linear07.html',
 'https://eman-physics.net/math/linear08.html',
 'https://eman-physics.net/math/linear09.html',
 'https://eman-physics.net/math/linear10.html',
 'https://eman-physics.net/math/linear11.html',
 'https://eman-physics.net/math/linear12.html',
 'https://eman-physics.net/math/linear13.html',


In [11]:
from langchain_community.document_loaders import WebBaseLoader

# Load, chunk and index the contents of the textbook.
loader = WebBaseLoader(
    web_paths=tuple(links),  # Provide the list of URLs as a tuple
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("p")  # Parse only  tags for content extraction
    ),
)

docs = loader.load()

In [26]:
# テキストを小さなチャンクに分割
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
    )
texts = text_splitter.split_documents(docs)

In [29]:
a = texts[80].page_content
print(a)

線形従属であるようなベクトルの集まりから幾つかのベクトルをうまく選んで捨てることで,線形独立なベクトルの集まりにすることが出来る.余分だったものを捨てるわけだ.この時,線形独立なベクトルを最大で幾つ残すことができるかを表しているのがランクであるとも言えるわけだ.
ここまでは「行列の中に含まれる各列をベクトルの成分だとみなした場合に」などという表現が繰り返されているが,列ではなく行の方をベクトルの成分だとみなして考えてはいけないのだろうか？
例えば次のような連立方程式を考える.
これらの式がそれぞれに独立な意味を持っているかどうか,ということが気になることがあると思う.独立でなければ解が一通りに定まらなかったり「解なし」ということになったりするだろう.
しかしそういう事を考えているとき,これらの式から係数を抜き出して作った次のような行列の列の方ではなく,各行の成分の方を「ベクトルに似た何か」として見ているようなものである.
そういう考え方をしても問題はないだろうか？
結論としては問題ない.列を取り出してベクトルとして考えてきたのは幾何学的な変換のイメージから話を進めた都合である.行列式の計算については「行で成り立つことは列についてもそのまま成り立っている」のだった.だから列と行を入れ替えたとしても最終的な値は変らない.またランクを求める過程についても,列への操作と行への操作は,基本変形行列を右から掛けるか左から掛けるかの違いだけなので,どちらにしても答えは変らない.
このことを式で表わしておこうか.式を使って証明しようというわけではない.今話したことをまとめ直すだけである.数式で表現されているだけで安心して受け入れられるという人は割りと多いからね.
先ほどの行列の中の各行を列にして書き直すと次のようになる.
これをと書いたのは,行列の転置行列という意味である.すでに余因子行列のところで軽く説明したことがあるが,もう一度説明しておこう.「転置行列」というのは行列の中の成分をの位置に置き換えたものだ.正方行列の左上から右下に線を引いて,その線を対称線として中身を入れ替えた形になる.今回のように行と列の役割を入れ替えたものだと考えてもいい.ちなみに,行列の転置行列をさらに転置したものは元の行列と同じものである.ま,元に戻るだけなので当然のことだな.


In [87]:
# 2. ベクトルストアの作成
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(
    texts, 
    embeddings
    )

In [95]:
retriever = vectorstore.as_retriever()

In [96]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define a custom function to format sources from retrieved documents
def format_sources(docs):
    """
    Extracts and formats the sources (metadata) from the retrieved documents.
    This function ensures that the source information is passed to the LLM as part of the context.
    """
    # Extract sources (e.g., URLs or document titles) from metadata
    sources = [doc.metadata.get('source', 'Unknown Source') for doc in docs]
    # Join the sources into a single string to pass to the LLM
    return "Sources:\n" + "\n".join(sources)

In [88]:
# 3. GPT-4モデルの準備
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)


In [113]:
# PromptTemplate class is used to create a custom prompt template for the LLM
from langchain.prompts import PromptTemplate

# Prompt template (`<||>` denotes section; `<>` denotes placeholder):
# `<|system|>`: defines the system's role, provide general instructions to LLM
# `{context}`: a placeholder for the context retrieved from the knowledge base.
#   It's dynamically replaced with relevant information related to the user's query.
# `<|user|>`: represents the user's input, where the user asks a question.
# {question}: a placeholder which will be replaced with the user's actual question.
# `<|assistant|>`: This is where the language model (assistant) generates its response,
#   based on both the context and the question.
prompt_template = """
<|system|>
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
keep the answer as concise as possible.
Use markdown formatting when displaying code.
Emphasis should be used to terminologies.
You need to provide the sources of the information you provide.
Use mathematical notation when necessary.
Always say "thanks for asking!" at the end of the answer.

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

"""

#  Create the PromptTemplate Instance
prompt = PromptTemplate(
    input_variables=[
        "context",
        "question",
        "sources"
        ],
    template=prompt_template,
)

# Or using availible prompts
# from langchain import hub
# from langchain_chroma import Chroma

# prompt use
# prompt = hub.pull("rlm/rag-prompt"

In [114]:
# This is an output parser used to convert the raw output from the LLM into a string.
# It ensures the LLM’s output is formatted as readable text.
from langchain_core.output_parsers import StrOutputParser
# This is a utility that simply passes the input data through without any modifications.
# It’s used when you want to allow the data to flow through a pipeline step unchanged.
from langchain_core.runnables import RunnablePassthrough

# LLM chain:
# The prompt is sent to the LLM.
# The LLM generates a response.
# The generated response is passed to StrOutputParser for formatting.
llm_chain = prompt | llm | StrOutputParser()

# RAG chain:
# 1. `context`: contextual information retrieved from the knowledge base.
#     The value is a pipeline that first uses retriever to find relevant documents,
#     and then passes the retrieved documents through format_docs to format them for further use.
#     Additionally, the `format_sources` step is added to include the sources.
# 1.1. `retriever`: Retrieves relevant documents from the knowledge base based on the user's query.
# 1.2. `format_docs`: Formats the retrieved documents.
# 1.3. `format_sources`: Extracts the sources of the retrieved documents (e.g., URLs).
# 2. `question`: the user's input query, which is passed directly through RunnablePassthrough().
#     The RunnablePassthrough simply forwards the original question without any modification.
# 3. `llm_chain`: Takes these inputs (context + sources + question) and generates a response using the LLM.
rag_chain = (
    {
        "context": retriever | format_docs,    # Retrieval Step for context
        "sources": retriever | format_sources, # Retrieve and format sources
        "question": RunnablePassthrough()      # Prompt Generation
    }
    | llm_chain                                # Generation Step
)

In [115]:
# To  display the output (answer) in a neatly formatted Markdown format
# within a Jupyter notebook
from IPython.display import Markdown, display

def display_answer(answer):
    display(Markdown(answer))

In [117]:
# My question
query = "解析力学的に運動方程式を導出する方法を教えてください。"

In [118]:
# answer with the context from our vector DB
anwser = rag_chain.invoke(query)
display_answer(anwser)

運動方程式を解析力学的に導出するためには、ラグランジュの公式が一般的に用いられます。ラグランジュの公式は、運動の法則をエネルギーの観点から記述します。

まず、系の力学的エネルギー（運動エネルギーとポテンシャルエネルギーの和）を考えます。これを `L` とし、以下のように定義します。

```
L = T - V
```

ここで、`T` は運動エネルギー、`V` はポテンシャルエネルギーです。

次に、ラグランジュの公式を適用します。これは以下の微分方程式で表されます。

```
d/dt (∂L/∂q̇) - ∂L/∂q = 0
```

ここで、`q` は一般化座標（運動を記述するためのパラメータ）、`q̇` は一般化速度（一般化座標の時間微分）、`∂/∂q` と `∂/∂q̇` はそれぞれ `q` と `q̇` に関する偏微分を表します。

この微分方程式を解くことで、系の運動方程式を求めることができます。

以上が解析力学的な運動方程式の導出方法です。具体的な計算は問題の詳細な条件に依存しますので、具体的な問題がある場合にはそれに対応して適用してみてください。

情報源: [Wikipedia: Lagrangian mechanics](https://en.wikipedia.org/wiki/Lagrangian_mechanics)

Thanks for asking!

In [119]:
# answer without external knowledge
answer_without_knowledge = llm_chain.invoke({"context":"", "question": query})
display_answer(answer_without_knowledge)

解析力学における運動方程式の導出は、ラグランジアンやハミルトニアンなどの概念を用いて行われます。以下にその基本的な手順を説明します。

1. **ラグランジアンの定義**: ラグランジアンは、系の運動エネルギーとポテンシャルエネルギーの差で定義されます。数学的には、運動エネルギーを`T`、ポテンシャルエネルギーを`V`としたとき、ラグランジアン `L`は以下のように定義されます。

    ```math
    L = T - V
    ```

2. **運動方程式の導出**: ラグランジェの運動方程式は、ラグランジアンが最小となる経路をとるという原理（最小作用原理）から導出されます。数学的には、以下の微分方程式が得られます（`q`は一般化座標、`t`は時間）。

    ```math
    \frac{d}{dt} \left(\frac{\partial L}{\partial \dot{q}}\right) - \frac{\partial L}{\partial q} = 0
    ```

これらの手順を用いて、任意の物理系の運動方程式を解析力学的に導出することができます。

なお、解析力学は物理学の一部であり、一般には高等教育の一環として教えられます。この分野の理解には高度な数学的知識が必要ですので、詳しい理解には専門的な教育や自己学習が必要となります。

参考資料: 
- Goldstein, H., Poole, C., & Safko, J. (2001). Classical Mechanics (3rd ed.). Addison Wesley.
- Landau, L. D., & Lifshitz, E. M. (1976). Mechanics (3rd ed.). Butterworth-Heinemann.

ありがとう、ご質問いただき感謝します！